In [1]:
'''
Arquivos:
- Estabelecer a forma de guardar os arquivos conforme necessidade da CNN

Funções:
- Função de importação de arquivo com as seguintes colunas:
    Open High Low Close Volume + Indicadores
  Esta mesma função fará os índices das linhas serem as datas
- Função de escolha do intervalo de dias a terem o gráfico montado
- Função que gera o gráfico de candlestick com volume e indicadores

'''

'\nArquivos:\n- Estabelecer a forma de guardar os arquivos conforme necessidade da CNN\n\nFunções:\n- Função de importação de arquivo com as seguintes colunas:\n    Open High Low Close Volume + Indicadores\n  Esta mesma função fará os índices das linhas serem as datas\n- Função de escolha do intervalo de dias a terem o gráfico montado\n- Função que gera o gráfico de candlestick com volume e indicadores\n\n'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf
from datetime import datetime
from datetime import timedelta
from PIL import Image

In [3]:
'''
Importa os dados do ativo e renomeia as colunas
'''

file_name = 'BTOW3_1d.xlsx'
path = '../collected_ data/' + file_name

inplace = True
df_input = pd.read_excel(path)
df_input.dropna(inplace = inplace)
df_input.set_index('Data', inplace = inplace)
df_input.sort_index(axis = 0, inplace = inplace)
df_input.rename(inplace = inplace, columns = {
    'Abertura': 'Open',
    'Máxima': 'High',
    'Mínima': 'Low',
    'Fechamento': 'Close',
    'Volume Financeiro': 'Volume',
    'Média Móvel E [9]': 'MME009',
    'Média Móvel A [21]': 'MMA021',
    'Média Móvel A [200]': 'MMA200',
    'Média Móvel A [51]': 'MMA051',
    'Média Móvel A [80]': 'MMA080',
    'Média Móvel A [400]': 'MMA400',
    'Média Móvel E [400]': 'MME400'
})
df_input.head()

,Open,High,Low,Close,Volume,MME009,MMA021,MMA200,MMA051,MMA080,MMA400,MME400
Data,,,,,,,,,,,,
2006-11-03,45.11,45.90,44.04,45.15,19193744,43.41,41.81,46.50,43.11,42.18,36.51,36.51
2006-11-06,45.20,46.72,44.20,46.50,32347216,44.03,41.99,46.49,43.19,42.23,36.57,36.56
2006-11-07,46.50,47.51,45.46,47.50,43484428,44.72,42.34,46.49,43.30,42.31,36.64,36.62
2006-11-08,47.39,47.90,46.14,47.75,52434539,45.33,42.76,46.50,43.40,42.41,36.71,36.67
2006-11-09,47.82,48.15,45.80,46.05,18398881,45.47,43.04,46.50,43.45,42.51,36.77,36.72


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf
from datetime import timedelta strptime

In [4]:
'''
Cria reordena as colunas dos indicadores em ordem alfabética
'''

df_candle = df_input[['Open', 'High', 'Low', 'Close', 'Volume']]
df_ind = df_input.reindex(sorted(df_input.columns[5:]), axis = 1)
df = pd.concat([df_candle, df_ind], axis=1)
df.head()

,Open,High,Low,Close,Volume,MMA021,MMA051,MMA080,MMA200,MMA400,MME009,MME400
Data,,,,,,,,,,,,
2006-11-03,45.11,45.90,44.04,45.15,19193744,41.81,43.11,42.18,46.50,36.51,43.41,36.51
2006-11-06,45.20,46.72,44.20,46.50,32347216,41.99,43.19,42.23,46.49,36.57,44.03,36.56
2006-11-07,46.50,47.51,45.46,47.50,43484428,42.34,43.30,42.31,46.49,36.64,44.72,36.62
2006-11-08,47.39,47.90,46.14,47.75,52434539,42.76,43.40,42.41,46.50,36.71,45.33,36.67
2006-11-09,47.82,48.15,45.80,46.05,18398881,43.04,43.45,42.51,46.50,36.77,45.47,36.72


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3400 entries, 2006-11-03 to 2020-08-07
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    3400 non-null   float64
 1   High    3400 non-null   float64
 2   Low     3400 non-null   float64
 3   Close   3400 non-null   float64
 4   Volume  3400 non-null   int64  
 5   MMA021  3400 non-null   float64
 6   MMA051  3400 non-null   float64
 7   MMA080  3400 non-null   float64
 8   MMA200  3400 non-null   float64
 9   MMA400  3400 non-null   float64
 10  MME009  3400 non-null   float64
 11  MME400  3400 non-null   float64
dtypes: float64(11), int64(1)
memory usage: 345.3 KB


In [6]:
'''
Verifica se o dataframe reordenado é igual ao original
'''

columns = list(df.columns)
eq = True
for col in columns:
   eq = eq*df[col].equals(df_input[col])
if eq:
    print("Data Frame válido.")
else:
    print("Data Frame inválido!!!!")

Data Frame válido.


In [7]:
'''
Aprender a gerar os gráficos para então iterar os intervalos de data
'''

def plot_chart(df, start_date, end_date, add):
    df_plot = df.loc[start_date:end_date,:]
    erase_color = 'black'

    mc = mpf.make_marketcolors(up = 'g', down = 'r', wick = 'inherit', edge = 'inherit', volume = 'silver')
    s = mpf.make_mpf_style(marketcolors = mc, facecolor = erase_color, edgecolor = erase_color, figcolor = erase_color)
    addplot = mpf.make_addplot(df_plot[add])
    
    return mpf.plot(df_plot, type = 'candle', volume = True, style = s, addplot = addplot, savefig = '../generated_data/teste.png')

ind_columns = list(df.columns)[5:]
start_date = '2020-01-01'
date_delta = 65
end_date = datetime.strptime(start_date, '%Y-%m-%d') + timedelta(days = date_delta)
end_date = end_date.strftime('%Y-%m-%d')

plot = plot_chart(df, start_date, end_date, ind_columns)

In [8]:
# Implementar o crop na função plot_chat()

box = (170, 65, 695, 478)

# Para salvar em PNG
im = Image.open('../generated_data/teste.png')
im_crop = im.crop(box).save('../generated_data/teste_crop.png')

# Para salvar em JPG
# im = Image.open('../generated_data/teste.png').convert('RGB')
# im_crop = im.crop(box).save('../generated_data/teste_crop.jpg', quality = 95)